# Tabula Muris Train

Train a tissue classifier on [Tabula Muris](http://tabula-muris.ds.czbiohub.org/) as per the paper:

[Learning a hierarchical representation of the yeast transcriptomic machinery using an autoencoder model](https://www.ncbi.nlm.nih.gov/pubmed/26818848)

Other References:

http://tiao.io/posts/implementing-variational-autoencoders-in-keras-beyond-the-quickstart-tutorial/

Read and write exclusively using S3 so that this notebook converted to python can be run in a kubernettes cluster for distributed machine learning.

In [6]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf

np.random.seed(42)  # reproducibility

# Simple syntatic sugar for debug vs. train parameters
def debug(debug_param, no_debug_param):
    return debug_param if os.environ.get("DEBUG") else no_debug_param
print(debug("DEBUG ON", "DEBUG OFF"))

tf.logging.set_verbosity(tf.logging.ERROR)

bucket_name = "stuartlab"
project_name = "tabula-muris"  # Dataset folder and output location

DEBUG ON


In [2]:
import boto3
session = boto3.session.Session(profile_name=os.getenv("AWS_PROFILE"))
bucket = session.resource(
    "s3", endpoint_url=os.getenv("AWS_S3_ENDPOINT")).Bucket(bucket_name)
print("S3 Profile: {} Endpoint: {} Project: {}".format(
    os.getenv("AWS_PROFILE"), os.getenv("AWS_S3_ENDPOINT"), project_name))

metadata = json.loads(bucket.Object(
    project_name + "/FACS/metadata.json").get()['Body'].read().decode('utf-8'))
print("Dataset metadata keys:", list(metadata.keys()))

S3 Profile: prp Endpoint: https://s3.nautilus.optiputer.net Project: tabula-muris
Dataset metadata keys: ['num_test_samples', 'num_train_samples', 'genes', 'tissues']


In [3]:
# Initialize model params - we'll store final hyper parameters here as well
params = {"batch_size": 128}

In [4]:
# Create a dataset that supports distributed training efficiently
def parse_one_example(example):
    features = {
        "features": tf.FixedLenFeature([len(metadata["genes"])], tf.float32),
        "labels": tf.FixedLenFeature([], tf.int64)
    }          
    example = tf.parse_single_example(example, features)
    return example["features"], tf.one_hot(example["labels"], len(metadata["tissues"]))

def create_dataset(files, batch_size, num_classes):
    # Assuming one file per class we set the cycle length to the
    # total number of classes so that our batches stratify accross all
    # classess and then shuffle
    files = tf.data.Dataset.list_files(files)
    dataset = files.interleave(lambda x: tf.data.TFRecordDataset(x, compression_type="GZIP").prefetch(64), 
                               cycle_length=num_classes, block_length=16*num_classes)

    # Use num_parallel_calls to parallelize map().
    dataset = dataset.map(parse_one_example, num_parallel_calls=num_classes)
    dataset = dataset.shuffle(16*num_classes)
    dataset = dataset.repeat(100)
    dataset = dataset.batch(batch_size)

    # Use prefetch() to overlap the producer and consumer.
    dataset = dataset.prefetch(1)
    return dataset

# Find all the training files
training_files = ["s3://{}/{}".format(bucket_name, o.key) 
         for o in bucket.objects.filter(Prefix=project_name + "/FACS/") 
         if o.key.endswith("train.gzip.tfrecord")]
print("Found {} training tfrecord files".format(len(training_files)))
training_dataset = create_dataset(training_files, params["batch_size"], len(training_files))

# Find all the test files
testing_files = ["s3://{}/{}".format(bucket_name, o.key) 
         for o in bucket.objects.filter(Prefix=project_name + "/FACS/") 
         if o.key.endswith("test.gzip.tfrecord")]
print("Found {} testing tfrecord files".format(len(testing_files)))
test_dataset = create_dataset(testing_files, params["batch_size"], len(testing_files))

Found 18 training tfrecord files
Found 18 testing tfrecord files


In [5]:
def create_model(input_size, output_size, hyperparameters={"width": 64, "depth": 2, "penalty": 1e-5}):
    input_layer = tf.keras.Input(shape=(input_size, ), name="input_layer")

    x = tf.keras.layers.BatchNormalization()(input_layer)
        
    for i in range(hyperparameters["depth"]):
        x = tf.keras.layers.Dense(hyperparameters["width"],
                                  activity_regularizer=tf.keras.regularizers.l1(
                                     hyperparameters["penalty"]), activation="relu")(x)
        x = tf.keras.layers.Dropout(0.5)(x)

    output_layer = tf.keras.layers.Dense(output_size, activation="softmax")(x)
    
    return tf.keras.Model(inputs=input_layer, outputs=output_layer)

model = create_model(input_size=len(metadata["genes"]), output_size=len(metadata["tissues"]))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],
              optimizer=tf.contrib.keras.optimizers.Adam())

model.summary()

model.fit(training_dataset, epochs=debug(1, 10),
          steps_per_epoch=debug(10, metadata["num_train_samples"] // params["batch_size"]),
          callbacks=[tf.keras.callbacks.TensorBoard(log_dir="/tmp")])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 23433)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 23433)             93732     
_________________________________________________________________
dense (Dense)                (None, 64)                1499776   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 18)                1170      
Total para

In [6]:
model.evaluate(test_dataset, 
               steps=debug(10, metadata["num_test_samples"] // params["batch_size"]))                                                                

10/10 [==============================] - 4s 436ms/step


[29.101637601852417, 0.2640625]

In [7]:
# Delete all s3 files so we start clean
# !aws s3 rm --recursive s3://stuartlab/tabula-muris/models --profile prp --endpoint https://s3.nautilus.optiputer.net

In [8]:
# Save the params, model and weights to S3 for evaluation back on jupyter
bucket.Object("{0}/models/{0}.params.json".format(project_name)).put(Body=json.dumps(params), ACL="public-read")

import tempfile
tempname = next(tempfile._get_candidate_names())
model.save("/tmp/{}.h5".format(tempname))
bucket.Object("{0}/models/{0}.model.h5".format(project_name)).upload_file(
    "/tmp/{}.h5".format(tempname), ExtraArgs={"ACL":"public-read"})
os.remove("/tmp/{}.h5".format(tempname))

In [9]:
# !aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
#     s3 ls s3://stuartlab/tabula-muris/models/

2018-10-10 21:44:13   18847136 tabula-muris.model.h5
2018-10-10 21:44:12         19 tabula-muris.params.json
